In [1]:
import pandas as pd
import numpy as np
import os
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/Users/ngochoang/Library/CloudStorage/GoogleDrive-nhungoc1508@gmail.com/My Drive/Graduate/Semester 1 (Fall 2024)/INFO-H423 Data Mining/Project/SNCB_IncidentClassifier/models/anomaly_detection'
os.chdir(path)

In [ ]:
df = pd.read_csv('../../data/time_sorted_table.csv', delimiter=';', index_col=0)
df.head(1)

,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
0,4432881,"[609, 609, 609, 609, 609, 609, 609, 609, 609, ...","[2744, 4004, 2852, 4110, 2854, 4396, 1132, 414...","[-5510, -5510, -5507, -5507, -5506, -5506, -55...",50.876601,4.718143,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal...",4


In [ ]:
import sys
import pandas as pd
from math import log
from collections import Counter
from ast import literal_eval

array_cols = ['seconds_to_incident_sequence', 'vehicles_sequence', 'events_sequence',
                     'train_kph_sequence', 'dj_ac_state_sequence', 'dj_dc_state_sequence']


def arrify_string_columns(df: pd.DataFrame):
    for col in array_cols:
        df[col] = df[col].apply(literal_eval)


def deep_copy(df: pd.DataFrame, additional_columns = []) -> pd.DataFrame:
    cols_to_copy = list.copy(array_cols)
    copy_df = df.copy(deep=True)
    cols_to_copy.extend(additional_columns)
    cols_to_copy = set(cols_to_copy)

    for col in cols_to_copy:
        if col in df:
            copy_df[col] = df[col].apply(list.copy)

    return copy_df


def get_frequency(list):
    element_counts = Counter(list)
    total_elements = len(list)
    return {element: count / total_elements for element, count in element_counts.items()}


def calculate_tfidf(events_column, dumping_function):
    events = []
    count = Counter()
    for events in events_column:
        count.update(set(events))

    number_of_documents_with_event = dict(count)
    number_of_rows = len(events_column)

    frequencies = events_column.apply(get_frequency)

    tfidf = {}
    for event in number_of_documents_with_event.keys():
        idf = log(number_of_rows/ number_of_documents_with_event[event])
        tfidf[event] = frequencies.apply(lambda x: idf * dumping_function(x.get(event, 0)))

    return tfidf

In [ ]:
arrify_string_columns(df)

In [ ]:
embed_dict = calculate_tfidf(df['events_sequence'], lambda x: math.log(1+x))

In [ ]:
vocab_path = './embeddings/metadata.tsv'
vocab = pd.read_csv(vocab_path, delimiter='\t', header=None)
vocab.columns = ['words']
vocab.head()

,words
0,[PAD]
1,[UNK]
2,2956
3,3658
4,3636


In [ ]:
embed_len = list(embed_dict.values())[0].shape[0]

def tfidf_lookup(word):
    if word not in ['[PAD]', '[UNK]']:
        return embed_dict[int(word)].to_list()
    else:
        return [0] * embed_len

In [ ]:
vocab['embed'] = vocab.apply(lambda x: tfidf_lookup(x.words), axis=1)
vocab.head()

,words,embed
0,[PAD],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,[UNK],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2956,"[0.0, 0.07554704357998568, 0.0, 0.0, 0.0, 0.05..."
3,3658,"[0.0, 0.007127718645050187, 0.0065821222316653..."
4,3636,"[0.0, 0.0073795892729018485, 0.006814713238920..."


In [ ]:
with open('./embeddings/tfidf.tsv', 'w') as output_file:
    for i, row in vocab.iterrows():
        embed = [str(n) for n in row['embed']]
        output_file.write('\t'.join(embed) + '\n')